In [1]:
import pymc as pm
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Supponiamo che il modello sia y ~ Normal(mu, sigma)
def model_H(data):
    with pm.Model() as model:
        mu = pm.Normal("mu", mu=0, sigma=10)
        sigma = pm.HalfNormal("sigma", sigma=5)
        y_obs = pm.Normal("y", mu=mu, sigma=sigma, observed=data)
    return model

def discrepancy(y, mu, sigma):
    return np.sum((y - mu)**2 / sigma**2)  # tipo chi²

In [6]:
# Step 1: campiona posteriore con y1
model1 = model_H(y1)
with model1:
    trace1 = pm.sample(1000, tune=1000, return_inferencedata=False)

NameError: name 'y1' is not defined

In [2]:

# Step 2: genera y2_rep usando posterior di y1
D_y2_given_y1 = []
D_y2rep_given_y1 = []

for mu, sigma in zip(trace1["mu"], trace1["sigma"]):
    y2_rep = np.random.normal(mu, sigma, size=len(y2))
    D_y2 = discrepancy(y2, mu, sigma)
    D_y2rep = discrepancy(y2_rep, mu, sigma)
    D_y2_given_y1.append(D_y2)
    D_y2rep_given_y1.append(D_y2rep)
# Step 3: calcola p-value predittiva
p_value = np.mean(np.array(D_y2rep_given_y1) > np.array(D_y2_given_y1))
print(f"Posterior predictive p-value of y2 using posterior of y1: {p_value:.3f}")
# Step 4: visualizza
plt.hist(D_y2rep_given_y1, bins=30, alpha=0.7, label='D(y2_rep; θ ∼ post(y1))')
plt.axvline(np.mean(D_y2_given_y1), color='red', linestyle='--', label='D(y2; θ ∼ post(y1))')
plt.legend()
plt.title('Posterior Predictive Discrepancy Check')
plt.xlabel('Discrepancy')
plt.ylabel('Frequency')
plt.show()

NameError: name 'y1' is not defined